In [124]:
import requests
import json

from openai import OpenAI
from langchain_openai import OpenAI, ChatOpenAI, OpenAIEmbeddings
from langchain.prompts import PromptTemplate, ChatPromptTemplate
from langchain.chains import LLMChain

from langchain_community.document_loaders import TextLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import Chroma
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser


In [125]:
import os
from dotenv import load_dotenv

In [126]:
load_dotenv()
openai_api_key = os.environ.get("OPENAI_API_KEY")

## Import Prompt

create new LLM, insert prompt and start chain

In [127]:
with open("prompt_continuouseffects.txt", "r") as file:
    prompt_continuous_effects = file.read()

In [128]:
print(prompt_continuous_effects)

"""
You will get two cards from the cardgame Magic: the Gathering. {card1} and {card2} both have continuous effects. You need to determine which effect will have which impact on the game.
You can assume that {card1} is played first, therefor has the earlier timestamp.
Use the {rules_db} as context for your decision.
You will get a {question} about two cards.

You will follow these steps:

1. Give the oracle texts from both cards.
2. Give your judging: The continuous effect of which card is applied in game: {card1} or {card2}?
3. Explain why you decided this way.
4. Quote the rule you used for your decision.

"""


## RAG
### Loader

In [129]:
loader = TextLoader("./data/rules_shortened.txt")
rules_doc = loader.load()

### Splitter

In [130]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
rules_splits = text_splitter.split_documents(rules_doc)
print(rules_splits[0])

page_content='611. Continuous Effects\n\n611.1. A continuous effect modifies characteristics of objects, modifies control of objects, or affects players or the rules of the game, for a fixed or indefinite period.\n\n611.2. A continuous effect may be generated by the resolution of a spell or ability.\n\n611.2a A continuous effect generated by the resolution of a spell or ability lasts as long as stated by the spell or ability creating it (such as “until end of turn”). If no duration is stated, it lasts until the end of the game.' metadata={'source': './data/rules_shortened.txt'}


### Vector Store

In [131]:
rules_db = Chroma.from_documents(documents=rules_splits, embedding=OpenAIEmbeddings())
retriever = rules_db.as_retriever()

- create new LLM
- load template

In [132]:
#llm = OpenAI(openai_api_key=openai_api_key)
model = ChatOpenAI(openai_api_key=openai_api_key)


In [133]:
#prompt_template = PromptTemplate.from_template(prompt_continuous_effects)
prompt_template = ChatPromptTemplate.from_template(prompt_continuous_effects)

## Case 1: Blood Moon vs. Urborg, Tomb of Yawgmoth

Get card information via scryfall.com and delete unnecessary data: 

In [134]:
card1 = "Blood Moon"
response = requests.get(f'https://api.scryfall.com/cards/named?exact={card1} ', 
                 headers={'Accept': 'application/json'})
if response.status_code == 200:
    # Parse the JSON response into a Python dictionary
    data = response.json()
    keywords = ["id", "oracle_id", "multiverse_ids", "mtgo_id", "tcgplayer_id", "cardmarket_id", "lang", "released_at", "uri", "scryfall_uri", "layout", "highres_image", "image_status", "image_uris", "legalities", "games", "reserved", "foil", "nonfoil", "finishes", "oversized", "promo", "reprint", "variation", "set_id", "set", "set_name", "set_type", "set_uri", "set_search_uri", "scryfall_set_uri", "rulings_uri", "prints_search_uri", "collector_number", "digital", "flavor_text", "card_back_id", "artist", "artist_ids", "illustration_id", "border_color", "frame", "security_stamp", "full_art", "textless", "booster", "story_spotlight", "edhrec_rank", "preview", "related_uris", "prices", "tcgplayer_infinite_articles", "tcgplayer_infinite_decks", "edhrec", "purchase_uris", "cardmarket", "cardhoarder"]
    for keyword in keywords:
        if keyword in data:
            del data[keyword]
    
    clean_json = json.dumps(data)
    # convert into json dict:
    card1 = json.loads(clean_json)
    print(card1)


{'object': 'card', 'name': 'Blood Moon', 'mana_cost': '{2}{R}', 'cmc': 3.0, 'type_line': 'Enchantment', 'oracle_text': 'Nonbasic lands are Mountains.', 'colors': ['R'], 'color_identity': ['R'], 'keywords': [], 'rarity': 'rare'}


In [135]:
card2 = "Urborg, Tomb of Yawgmoth"
response = requests.get(f'https://api.scryfall.com/cards/named?exact={card2} ', 
                 headers={'Accept': 'application/json'})
if response.status_code == 200:
    # Parse the JSON response into a Python dictionary
    data = response.json()
    keywords = ["id", "oracle_id", "multiverse_ids", "mtgo_id", "tcgplayer_id", "cardmarket_id", "lang", "released_at", "uri", "scryfall_uri", "layout", "highres_image", "image_status", "image_uris", "legalities", "games", "reserved", "foil", "nonfoil", "finishes", "oversized", "promo", "reprint", "variation", "set_id", "set", "set_name", "set_type", "set_uri", "set_search_uri", "scryfall_set_uri", "rulings_uri", "prints_search_uri", "collector_number", "digital", "flavor_text", "card_back_id", "artist", "artist_ids", "illustration_id", "border_color", "frame", "security_stamp", "full_art", "textless", "booster", "story_spotlight", "edhrec_rank", "preview", "related_uris", "prices", "tcgplayer_infinite_articles", "tcgplayer_infinite_decks", "edhrec", "purchase_uris", "cardmarket", "cardhoarder"]
    for keyword in keywords:
        if keyword in data:
            del data[keyword]
    
    clean_json = json.dumps(data)
    # convert into json dict:
    card2 = json.loads(clean_json)
    print(card2)

{'object': 'card', 'name': 'Urborg, Tomb of Yawgmoth', 'mana_cost': '', 'cmc': 0.0, 'type_line': 'Legendary Land', 'oracle_text': 'Each land is a Swamp in addition to its other land types.', 'colors': [], 'color_identity': [], 'keywords': [], 'produced_mana': ['B'], 'rarity': 'rare', 'frame_effects': ['legendary']}


In [136]:
chain = ({"card1": card1, "card2": card2, "rules_db": retriever, "question": RunnablePassthrough()} | prompt_template | model | StrOutputParser())
chain.invoke(f"What happens if player a plays {card1} and player b plays {card2}?")

TypeError: Expected a Runnable, callable or dict.Instead got an unsupported type: <class 'str'>

In [ ]:
# cleanup
rules_db.delete_collection()

## Old Version w/o RAG

In [ ]:
prompt = prompt_template.invoke({"card1": "Blood Moon", "card2": "Tomb of Yawgmoth", "rules_db": rules_db})
llm.invoke(prompt)

## Case 2: Opalescence vs Humility

In [ ]:
prompt_val = prompt_template.invoke({"card1": "Opalescence", "card2": "Humility"})

In [ ]:
llm.invoke(prompt)

In [ ]:
prompt = prompt_template.invoke({"card1": "Humility", "card2": "Opalescence"})

In [ ]:
llm.invoke(prompt)